In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(filepath_or_buffer='iris_data.csv', delimiter=",")
df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   target        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [4]:
def clean_data(df: pd.DataFrame):
    print('before droping the null valued columns')
    print(df.isna().sum())
    df = df.dropna()
    print('after droping the null valued columns')
    print(df.isna().sum())
    return df

In [5]:
df = clean_data(df)

before droping the null valued columns
sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
target          0
dtype: int64
after droping the null valued columns
sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
target          0
dtype: int64


In [6]:
def split_data(df: pd.DataFrame):
    from sklearn.model_selection import train_test_split

    X = df.loc[:, df.columns != 'target']
    y = df.loc[:, df.columns == 'target']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

    print('X train size:', X_train.shape)
    print('Y train size:', X_test.shape)
    print('X test size:', y_train.shape)
    print('Y test size:', y_test.shape)

    return X_train, X_test, y_train, y_test

In [7]:
X_train, X_test, y_train, y_test = split_data(df)

X train size: (105, 4)
Y train size: (45, 4)
X test size: (105, 1)
Y test size: (45, 1)


In [8]:
def basic_decision_tree_classifier(X_train, y_train):
    from sklearn.tree import DecisionTreeClassifier

    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)

    return model

In [9]:
model = basic_decision_tree_classifier(X_train, y_train)

In [10]:
def predict(model, X_test):
    y_predicted = model.predict(X_test)
    return y_predicted

In [11]:
def predict_probability(model, X_test):
    y_predicted_prob = model.predict_proba(X_test)
    return y_predicted_prob

In [12]:
def generate_metrics(y_predicted, y_test):
    from sklearn.metrics import accuracy_score, precision_score, recall_score

    # average param = [None, 'micro', 'macro', 'weighted'].

    acc = accuracy_score(y_test, y_predicted)
    prec = precision_score(y_test, y_predicted, average='macro')
    recall = recall_score(y_test, y_predicted, average='macro')
    return {'accuracy': round(acc, 2), 'precission': round(prec, 2), 'recall': round(recall, 2)}

In [13]:
feature_importance_df = pd.DataFrame(model.feature_importances_)
feature_importance_df.head(10)

,0
0,0.021469
1,0.021469
2,0.063170
3,0.893892


In [14]:
y_predicted = predict(model, X_test)
y_predicted_prob = predict_probability(model, X_test)
result = generate_metrics(y_predicted, y_test)
print(result)

{'accuracy': 0.96, 'precission': 0.96, 'recall': 0.96}


## MLflow starts

In [17]:
experiment_name = "basic_decision_tree_classifier"  #basic classifier
run_name = "iris_flower_classification"
run_metrics = generate_metrics(y_predicted, y_test)
print(run_metrics)

{'accuracy': 0.96, 'precission': 0.96, 'recall': 0.96}


In [22]:
def start_experiment(experiment_name:str, run_name:str, run_metrics:dict, model, run_params=None, tag:dict=None, signature=None):
    import mlflow
    mlflow.set_tracking_uri("http://127.0.0.1:3000")
    mlflow.set_experiment(experiment_name=experiment_name)
    with mlflow.start_run(run_name=run_name):
        if run_params != None:
            for param in run_params:
                mlflow.log_param(param, run_params[param])

        for metric in run_metrics:
            mlflow.log_metric(metric, run_metrics[metric])

        #mlflow.sklearn.log_model(model, "model")
        #mlflow.sklearn.log_model(model, "model", registered_model_name="iris-classification")

        mlflow.sklearn.log_model(model, "model", registered_model_name="iris-classification", signature=signature)

        for key in tag.keys():
            mlflow.set_tag(key, tag.get(key))

        #mlflow.set_tag("tag1", tag)

    print('Run - %s is logged to Experiment - %s' % (run_name, experiment_name))

In [21]:
start_experiment(experiment_name, run_name, run_metrics, model, tag={"basic_experiment":"basic_decision_tree_classifier"})

#start_experiment(experiment_name, run_name, run_metrics, model, tag="basic_decision_tree_classifier")

Successfully registered model 'iris-classification'.
2023/04/18 19:04:42 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-classification, version 1


Run - iris_flower_classification is logged to Experiment - basic_decision_tree_classifier


Created version '1' of model 'iris-classification'.


In [23]:
param_dict = {
    'criterion': ["gini", "entropy", "log_loss"],
    'splitter': ["best", "random"],
    'max_depth': [None, 5, 10, 15, 20, 35, 30],
    'class_weight': [None, "balanced"]
}

def optimised_decision_tree_classifier(X_train, y_train):
    from sklearn.model_selection import GridSearchCV
    from sklearn.tree import DecisionTreeClassifier

    clf = DecisionTreeClassifier()
    gridsearch_cv = GridSearchCV(clf, param_grid=param_dict, cv=10, n_jobs=5)
    model = gridsearch_cv.fit(X_train, y_train)
    return gridsearch_cv, model

In [24]:
result, model = optimised_decision_tree_classifier(X_train, y_train)
print(result.best_estimator_)
print(result.best_score_)
print(result.best_params_)

DecisionTreeClassifier(criterion='log_loss', max_depth=30, splitter='random')
0.9709090909090909
{'class_weight': None, 'criterion': 'log_loss', 'max_depth': 30, 'splitter': 'random'}


In [25]:
y_predicted = predict(model, X_test)
y_predicted_prob = predict_probability(model, X_test)

In [26]:
experiment_name = "optimised_decision_tree_classifier"  #basic classifier
run_name = "iris_flower_classification"
metrics = generate_metrics(y_predicted, y_test)
print(metrics)

{'accuracy': 0.96, 'precission': 0.96, 'recall': 0.96}


In [27]:
from mlflow.models.signature import infer_signature

# infer the models input structure and output structure
signature = infer_signature(X_train, model.predict(X_test))
start_experiment(experiment_name=experiment_name, run_name=run_name, run_metrics=metrics, model=model,
                 run_params=param_dict, tag={"optimal_experiment":"optimised_decision_tree_classifier","advanced_exp":"optimised_decision_tree_classifier"}, signature=signature)

2023/04/18 19:08:01 INFO mlflow.tracking.fluent: Experiment with name 'optimised_decision_tree_classifier' does not exist. Creating a new experiment.
Registered model 'iris-classification' already exists. Creating a new version of this model...
2023/04/18 19:08:02 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-classification, version 2


Run - iris_flower_classification is logged to Experiment - optimised_decision_tree_classifier


Created version '2' of model 'iris-classification'.


## Model Consuming

In [38]:
import mlflow

logged_model = 'runs:/16d6975f697c442497ac392972b53333/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd

predicted = loaded_model.predict(pd.DataFrame(X_test))
print(predicted[:10])

['Iris-setosa' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-virginica' 'Iris-versicolor' 'Iris-virginica' 'Iris-setosa'
 'Iris-setosa' 'Iris-virginica']


In [39]:
import mlflow

logged_model = 'runs:/49852a760bbf40c78da729b925fc8557/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd

predicted = loaded_model.predict([[2.3, 6.5, 4.4, 5.6]])
print(predicted)


['Iris-versicolor']


/Users/pavanmantha/miniconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [43]:
import mlflow.pyfunc as pyfunc
from mlflow.sklearn import load_model
model_name = "iris-classification"
model_version = 1

model = pyfunc.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)

y_pred = model.predict(X_test)
print(y_pred)

sklearn_model = load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)
y_pred_prob = sklearn_model.predict_proba(X_test)
print(y_pred_prob)

['Iris-setosa' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-virginica' 'Iris-versicolor' 'Iris-virginica' 'Iris-setosa'
 'Iris-setosa' 'Iris-virginica' 'Iris-versicolor' 'Iris-setosa'
 'Iris-virginica' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa' 'Iris-setosa'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-virginica' 'Iris-setosa'
 'Iris-virginica' 'Iris-versicolor' 'Iris-setosa' 'Iris-setosa'
 'Iris-versicolor' 'Iris-virginica' 'Iris-versicolor' 'Iris-virginica'
 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica' 'Iris-setosa'
 'Iris-versicolor' 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica'
 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica'
 'Iris-versicolor']
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1

## Push the model to production version

In [42]:
client = mlflow.tracking.MlflowClient()
client.transition_model_version_stage(
    name="iris-classification",
    version=1,
    stage="Staging"
)

<ModelVersion: creation_timestamp=1681824882465, current_stage='Staging', description='', last_updated_timestamp=1681826202501, name='iris-classification', run_id='49852a760bbf40c78da729b925fc8557', run_link='', source='/Users/pavanmantha/Pavans/PracticeExamples/DataScience_Practice/mlflow-tutorials/ml-classification-mlflow/mlruns/1/49852a760bbf40c78da729b925fc8557/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>